In [1]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))


In [34]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        
    }
    print(url)
    response = session.get(url, headers=headers)
    response.raise_for_status()
    if response.status_code != 200:
        raise Exception(f"[{response.status_code}] {response.text}")
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [52]:
sub_link_list = []

In [ ]:

import time

def get_link_list(url):
    soup = get_soup(url)
    for div in soup.find_all('div', class_="logobox"):
        a = div.find("a")
        if a:
            sub_link_list.append(a["href"])

get_link_list("https://www.sialchina.com/exhibitorlist.html")
for i in range(2, 298):
    get_link_list(f"https://www.sialchina.com/exhibitorlist.html?page={i}")
    time.sleep(0.3)

In [56]:
import json
with open('./output/sialchina_detail_page_link_list_tmp.json', '+wt') as fp:
    json.dump(sub_link_list, fp, ensure_ascii=False)

In [61]:
result_list = []

In [ ]:
from urllib.parse import urljoin

for i, url in enumerate(sub_link_list):
    sub_url = urljoin('https://www.sialchina.com', url)
    
    time.sleep(0.2)
    sub_soup = get_soup(sub_url)
    item = {}
    country_label = "Country/Region:"
    top_div = sub_soup.find('div', class_="txtfl")

    p_list = top_div.find_all("p")
    for p in p_list:
        txt = p.get_text(strip=True)
        if "company" in p["class"]:
            item["name"] = txt
        elif txt.startswith(country_label):
            item["country"] = txt.replace(country_label, '').strip()
            
    other_div = sub_soup.find('div', class_="otherInfo")
    if other_div:
        p_list = other_div.find_all("p")
        for p in p_list:
            if p.text.strip().startswith('By Product Category:'):
                if p.find('span'):
                    item["category"] = p.find('span').getText(strip=True)

            if p.text.strip().startswith('Brand Name:'):
                if p.find('a'):
                    item["website"] = p.find('a')["href"]


    result_list.append(item)
    
    

In [64]:
with open("./output/siachina_result_tmp.json", '+wt') as fp:
    json.dump(result_list, fp)

In [65]:
import pandas as pd

df = pd.DataFrame(result_list)
df_with_website = df.dropna(subset=['website'])
df_without_website = df[df['website'].isna()]

In [68]:
df_with_website.to_csv('./output/siachina_result_with_website.csv', index=False, quoting=1)
df_without_website.to_csv('./output/siachina_result_without_website.csv', index=False, quoting=1)

2524